In [2]:
import numpy as np
import magpylib as magpy
from magpylib.magnet import Cuboid, Cylinder, CylinderSegment
from maggeometry import halbach_cylinder
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import minimize, Bounds
from maghelper import get_field_on_axes, get_nonuniformity, make_flux_stream, centered_sweep_range, get_grid_nonuniformity, make_grid

### Nelder-Mead Adaptive Optimization of Halbach 9 main
n = 9

### Samwise

In [3]:
# def halbach_cylinder(Br, c_d, c_h, D, n, alternate=False):

In [4]:
def get_grid_mag_and_target_nonuniformity(magnets, grid, grid_res, target=10, use_z=False):
    mT_to_G = 10
    B = magnets.getB(grid)
    G = B * mT_to_G
    # find magnitude of the b-field at all points over grid
    Gmag = 0
    if use_z:
        Gmag = G[:,:,2]
    else:
        Gmag = np.linalg.norm(G, axis=2)
#     Gmag = np.sqrt(G[:,:,0]**2 + G[:,:,1]**2 + G[:,:,2]**2)
#     Gmag = G[:,:,2]
    # find magnitude of b-field at center of grid
    mid_id = int(grid_res/2)
    Gcenter = Gmag[mid_id][mid_id]
    # calculate nonuniformity with respect to center b-field
    Gnon = np.abs((Gmag - Gcenter)/Gcenter)
    Gtarget_non = np.abs((Gmag - target)/target)
#     Gtarget_non = np.abs((Gmag - target)/target)
#     max_nonuniformity = np.max(Gnon)
    av_nonuniformity = np.sum(Gnon) / grid_res**2
    av_target_nonuniformity = np.sum(Gtarget_non) / grid_res**2
#     return Gmag, Gnon, Gcenter, av_nonuniformity, av_target_nonuniformity, max_nonuniformity
    return Gcenter, av_nonuniformity, av_target_nonuniformity

In [5]:
from maghelper import get_field_on_axes, get_nonuniformity, make_flux_stream, centered_sweep_range, get_grid_nonuniformity, make_xy_grid

In [6]:
n = 9
grid_res = 101
grid = make_xy_grid([-10, 10], [-10, 10], grid_res)
Br = 1.09e3
c_h = 640
threshold = 0.1 # in G
target = 10

          #c_d,     D
bounds = ((1, 20), (21, 300))

def obj1_nonuniformity(x):
    magnets = halbach_cylinder(Br, x[0], c_h, x[1], n)
    center_field, av_nonuniformity, av_target_nonuniformity = get_grid_mag_and_target_nonuniformity(magnets, grid, grid_res, target)
    if (abs(center_field - target) > threshold):
        return av_target_nonuniformity
    return av_nonuniformity

In [ ]:
options = {'disp': True, 'fatol': 1e-18, 'maxiter': 5e3, 'adaptive': True}
method = "Nelder-Mead"

shots = 25
results_x = []
results_fun = []
guesses_x0 = []
b_len = len(bounds)
for i in range(shots):
    x0 = [0, 0]
    for b in range(b_len):
        x0[b] = np.random.uniform(low=bounds[b][0], high=bounds[b][1])
    print(x0)
    res = minimize(obj1_nonuniformity, x0, method=method, options=options, bounds=bounds)
    guesses_x0.append(x0)
    results_x.append(res.x)
    results_fun.append(res.fun)

print("Objective function values:")
print(results_fun)
print("Result parameters:")
print(results_x)
print("Starting guesses:")
print(guesses_x0)

[3.2163110975182834, 255.49538992700386]


/var/folders/06/7pjfbw510cz2ch1m7lx8vvl80000gn/T/ipykernel_20940/1123013417.py:14: RuntimeWarning: Maximum number of iterations has been exceeded.
  res = minimize(obj1_nonuniformity, x0, method=method, options=options, bounds=bounds)


[6.7739217983226006, 101.21539798347531]
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 160
         Function evaluations: 360
[6.004698406180351, 36.132024398383955]
[17.366592140468306, 217.51969260578076]
[15.460419657523563, 199.8240990924632]
[15.134877685425474, 118.21608702588321]
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 133
         Function evaluations: 296
[4.259018298946037, 229.58156494087555]
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 193
         Function evaluations: 424
[10.146756717268053, 224.97672415159383]
[10.338129254680299, 100.3723189669202]
[4.096284808009029, 41.311557583397686]
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 97
         Function evaluations: 229
[14.921613240231041, 31.594659791356975]
Optimization terminated successfully

In [7]:
print(results_x)

[array([ 1.10722033, 68.20068613]), array([ 1.10538425, 68.22645632]), array([ 1.10546849, 68.25113915]), array([ 1.10632423, 68.17203017]), array([ 1.1039182 , 68.20489546]), array([ 1.10528274, 68.17988064]), array([  4.91313203, 300.        ]), array([  4.91274976, 300.        ]), array([  4.91270058, 300.        ]), array([ 1.10464234, 68.18975325]), array([ 1.10495303, 68.17439025]), array([ 1.        , 62.07562626]), array([  4.91266973, 300.        ]), array([  4.91328816, 300.        ]), array([ 1.10123309, 68.22828601]), array([ 1.10589914, 68.24967072]), array([ 1.        , 62.07571181]), array([ 1.10409497, 68.178942  ]), array([  4.91203899, 300.        ]), array([ 1.10505445, 68.17585318]), array([ 1.10527117, 68.26592888]), array([ 1.10829653, 68.25493822]), array([ 1.        , 62.07557495]), array([ 1.10301058, 68.1812621 ]), array([ 1.10245568, 68.13842094])]


In [8]:
charr = np.full((1, shots), 640).flatten()
r_n = np.full((1, shots), n).flatten()

In [9]:
from maggeometry import halbach_cylinder
from maghelper import get_field_on_axes, get_nonuniformity, make_flux_stream, centered_sweep_range, get_grid_nonuniformity, make_xy_grid, get_grid_mag_and_nonuniformity
grid_res = 101
grid = make_xy_grid([-10, 10], [-10, 10], grid_res)
results_g_center = []
results_nonun = []
for x in results_x:
    magnets = halbach_cylinder(Br, x[0], charr[0], x[1], n)
    Gmag, Gnon, center_field, av_nonuniformity, max_abs_nonuniformity = get_grid_mag_and_nonuniformity(magnets, grid, grid_res)
    results_g_center.append(center_field)
    results_nonun.append(av_nonuniformity)

In [10]:
import pandas as pd
def make_opt_res_halbach_csv(name, fun, g_center, r_n, r_ch, x, guesses):
    r_f = np.array([fun])
    r_g = np.array(guesses)
    r_gc = np.array([g_center])
    r_n = np.array([r_n])
    r_ch = np.array([r_ch])
    r_x = np.array(x)
    a = np.concatenate((r_f.T, r_gc.T, r_n.T, r_ch.T, r_x, r_g), axis=1)
    b = a[a[:, 0].argsort()]

    columns = ['nonuniformity', 'center_field_gauss', 'n_rods', 'cyl_height']

    columns.append('cyl_diameter')
    columns.append('D')
    columns.append('g_cyl_diameter')
    columns.append('g_D')
    
    
    columns = np.array(columns)
    b_df = pd.DataFrame(b, columns = columns)
    
    c = np.concatenate((r_f.T, r_x), axis=1)
    new_array = [tuple(row) for row in c]
    u = np.unique(new_array, axis=0)
    n_unique = len(u)
    print('Number of unique results:', n_unique)
    print(b_df)
    b_df.to_csv(name + str(n_unique) + '.csv')
    return b, b_df

import datetime
today = datetime.datetime.today().strftime('%Y_%m_%d')

b, b_df = make_opt_res_halbach_csv(f'results/halbach/samwise/{n}_{today}_pippin1_halbach_main_1090mT_', results_fun, results_g_center, r_n, charr, results_x, guesses_x0)

Number of unique results: 25
    nonuniformity  center_field_gauss  n_rods  cyl_height  cyl_diameter  \
0    1.556855e-07            9.938103     7.0       640.0      1.101233   
1    1.556877e-07           10.058140     7.0       640.0      1.108297   
2    1.556879e-07            9.993477     7.0       640.0      1.103918   
3    1.556882e-07           10.004291     7.0       640.0      1.104095   
4    1.556907e-07           10.013706     7.0       640.0      1.105384   
5    1.556914e-07           10.021185     7.0       640.0      1.104953   
6    1.556917e-07           10.025551     7.0       640.0      1.105283   
7    1.556926e-07           10.011037     7.0       640.0      1.104642   
8    1.556931e-07           10.054594     7.0       640.0      1.107220   
9    1.556958e-07            9.983969     7.0       640.0      1.103011   
10   1.556965e-07           10.000081     7.0       640.0      1.105271   
11   1.556973e-07           10.007989     7.0       640.0      1.105468

In [12]:
m = halbach_cylinder(Br, 1.101233, 640, 68.228286, n)
Gmag, Gnon, center_field, av_nonuniformity, max_abs_nonuniformity = get_grid_mag_and_nonuniformity(m, grid, grid_res)
print(av_nonuniformity)

1.5573750827657786e-07
